# Проект: Анализ воронки оформления кредитной карты
## Выявление точек оттока для новых и существующих клиентов


## 🎯 **1. Бизнес-контекст и постановка задачи**


### **Бизнес-проблема**
В банке «ФинансГарант» заметили, что из всех посетителей страницы кредитных карт только 8% доходят до получения карты. Маркетинг тратит бюджет на привлечение клиентов, но они «отваливаются» на середине процесса. Особенно беспокоит низкая конверсия среди **новых клиентов** — тех, кто пришёл в банк впервые.

 **Гипотеза от продукт-менеджера:** «Новые клиенты не доверяют сложной форме с большим количеством полей и бросают процесс на этапе заполнения персональных данных».


### **Цель анализа**
1. Построить **воронку конверсии** из 6 ключевых этапов оформления карты.
2. Найти **этап с максимальным оттоком**.
3. Сравнить поведение **новых и существующих клиентов**.
4. Дать **рекомендации по оптимизации** процесса оформления.


### **Метрики успеха**
- **Конверсия между этапами** (в %) — сколько пользователей перешло с одного шага на следующий.
- **Общая конверсия** (view → issued) — ключевая метрика эффективности воронки.
- **Статистически значимые различия** между сегментами клиентов.


## 📊 **2. Загрузка данных и EDA (Exploratory Data Analysis)**

- Подключаем инструменты для работы с данными (pandas), математики (numpy) и визуализации (matplotlib, seaborn)  
- Настраиваем отображение, чтобы видеть полные данные  
- Устанавливаем приятные стили для графиков  

In [3]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Настройка отображения для удобства
pd.set_option('display.max_columns', None)  # Показывать все колонки
pd.set_option('display.width', None)        # Ширина вывода
pd.set_option('display.max_colwidth', None) # Не обрезать длинные тексты

# Настройка стилей графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

- Проверяем наличие файла с данными
- Загружаем CSV в DataFrame
- Парсим даты автоматически
- Выводим основную информацию о данных

In [2]:
# Загрузка данных
data_path = 'bank_data/credit_card_events.csv'

# Проверяем, существует ли файл
if not os.path.exists(data_path):
    print(f"❌ Файл {data_path} не найден!")
    print("Запустите сначала ноутбук '1_generate_data.ipynb'")
else:
    # Загружаем CSV файл
    df = pd.read_csv(data_path, parse_dates=['event_timestamp'])
    print(f"✅ Данные загружены успешно!")
    print(f"   Размер данных: {df.shape[0]:,} строк, {df.shape[1]} столбцов")
    print(f"   Период данных: {df['event_date'].min()} — {df['event_date'].max()}")

✅ Данные загружены успешно!
   Размер данных: 15,320 строк, 8 столбцов
   Период данных: 2025-10-31 — 2026-02-04


In [4]:
print("\n🔍 ПЕРВИЧНЫЙ ОСМОТР ДАННЫХ")
print("=" * 60)

# Показываем первые 5 строк
print("\nПервые 5 строк данных:")
print(df.head())

# Информация о типах данных и пропусках
print("\n\nИнформация о типах данных и пропусках:")
print(df.info())

# Базовая статистика по всем колонкам
print("\n\nБазовые статистики:")
print(df.describe(include='all').round(2))


🔍 ПЕРВИЧНЫЙ ОСМОТР ДАННЫХ

Первые 5 строк данных:
   user_id client_type  event_date            event_timestamp     event_name  \
0        0    existing  2025-11-26 2025-11-26 15:18:17.121169      page_view   
1        0    existing  2025-11-28 2025-11-28 14:41:17.121169    card_select   
2        0    existing  2025-11-28 2025-11-28 17:56:17.121169      app_start   
3        0    existing  2025-11-30 2025-11-30 08:29:17.121169      data_fill   
4        0    existing  2025-11-30 2025-11-30 12:55:17.121169  phone_confirm   

  device_type marketing_source  event_order  
0      mobile            email            1  
1      mobile            email            2  
2      mobile            email            3  
3      mobile            email            4  
4      mobile            email            5  


Информация о типах данных и пропусках:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtyp

**Что проверяем:**

| Проверка          | Что означает                       | Критерий качества             |
|-------------------|------------------------------------|-------------------------------|
| Пропуски          | Пустые значения в данных           | Должно быть 0%                |
| Дубликаты         | Полностью одинаковые строки        | Должно быть 0%                |
| Уникальные значения | Разнообразие данных               | Должно быть разумное количество |

In [5]:
print("\n\n✅ ПРОВЕРКА КАЧЕСТВА ДАННЫХ")
print("=" * 60)

# 1. Проверка пропусков
missing_values = df.isnull().sum()
print("\n1. Пропущенные значения:")
if missing_values.sum() == 0:
    print("   ✅ Нет пропущенных значений")
else:
    for col, count in missing_values[missing_values > 0].items():
        print(f"   ❌ {col}: {count} пропусков ({count/len(df):.1%})")

# 2. Проверка дубликатов
duplicates = df.duplicated().sum()
print(f"\n2. Полные дубликаты строк: {duplicates} ({duplicates/len(df):.1%})")

# 3. Проверка уникальных значений
print("\n3. Уникальные значения по столбцам:")
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"   {col}: {unique_count:,} уникальных значений")



✅ ПРОВЕРКА КАЧЕСТВА ДАННЫХ

1. Пропущенные значения:
   ✅ Нет пропущенных значений

2. Полные дубликаты строк: 0 (0.0%)

3. Уникальные значения по столбцам:
   user_id: 5,000 уникальных значений
   client_type: 2 уникальных значений
   event_date: 97 уникальных значений
   event_timestamp: 9,939 уникальных значений
   event_name: 6 уникальных значений
   device_type: 3 уникальных значений
   marketing_source: 5 уникальных значений
   event_order: 6 уникальных значений


**Вывод:** Данные качественные, можно приступать к анализу.

## **📋 3. Анализ воронки**

In [15]:
print("\n" + "="*50)
print("ШАГ 3.1: ОБЩАЯ ВОРОНКА (все пользователи)")
print("="*50)

# Считаем, сколько уникальных пользователей было на каждом этапе
events_order = ['page_view', 'card_select', 'app_start', 'data_fill', 'phone_confirm', 'card_issue']

print("\nКоличество пользователей на каждом этапе:")
for event in events_order:
    count = df[df['event_name'] == event]['user_id'].nunique()
    print(f"{event}: {count} пользователей")


ШАГ 3.1: ОБЩАЯ ВОРОНКА (все пользователи)

Количество пользователей на каждом этапе:
page_view: 5000 пользователей
card_select: 3800 пользователей
app_start: 2656 пользователей
data_fill: 1504 пользователей
phone_confirm: 1243 пользователей
card_issue: 1117 пользователей


**Вывод:** Видим стандартную воронку — чем дальше этап, тем меньше людей.  
Самый большой отток между app_start (2656) и data_fill (1504) — потеря 1152 пользователей.

In [18]:
print("\n" + "="*50)
print("ШАГ 3.2: ВОРОНКА ПО ТИПАМ КЛИЕНТОВ")
print("Что ищем: Различия в поведении новых и существующих клиентов")
print("="*50)

# Считаем реальное количество новых и существующих
total_new = df[df['client_type'] == 'new']['user_id'].nunique()
total_existing = df[df['client_type'] == 'existing']['user_id'].nunique()

print(f"\nНОВЫЕ клиенты (всего новых: {total_new}):")
new_clients = df[df['client_type'] == 'new']
for event in events_order:
    count = new_clients[new_clients['event_name'] == event]['user_id'].nunique()
    percentage = count / total_new * 100
    print(f"{event}: {count} ({percentage:.1f}%)")

print(f"\nСУЩЕСТВУЮЩИЕ клиенты (всего существующих: {total_existing}):")
existing_clients = df[df['client_type'] == 'existing']
for event in events_order:
    count = existing_clients[existing_clients['event_name'] == event]['user_id'].nunique()
    percentage = count / total_existing * 100
    print(f"{event}: {count} ({percentage:.1f}%)")


ШАГ 3.2: ВОРОНКА ПО ТИПАМ КЛИЕНТОВ
Что ищем: Различия в поведении новых и существующих клиентов

НОВЫЕ клиенты (всего новых: 1981):
page_view: 1981 (100.0%)
card_select: 1374 (69.4%)
app_start: 826 (41.7%)
data_fill: 321 (16.2%)
phone_confirm: 238 (12.0%)
card_issue: 201 (10.1%)

СУЩЕСТВУЮЩИЕ клиенты (всего существующих: 3019):
page_view: 3019 (100.0%)
card_select: 2426 (80.4%)
app_start: 1830 (60.6%)
data_fill: 1183 (39.2%)
phone_confirm: 1005 (33.3%)
card_issue: 916 (30.3%)


**Вывод:** Разница еще больше!   
У новых 16.1% доходят до data_fill, у существующих 39.4% — в 2.4 раза лучше.

In [19]:
print("\n" + "="*50)
print("ШАГ 3.3: КОНВЕРСИЯ НА ЭТАПЕ app_start → data_fill")
print("="*50)

# Берем цифры из предыдущего шага
app_start_new = 826
data_fill_new = 321

app_start_existing = 1830
data_fill_existing = 1183

# Считаем конверсию
conv_new = data_fill_new / app_start_new * 100
conv_existing = data_fill_existing / app_start_existing * 100

print(f"\nНОВЫЕ клиенты:")
print(f"Начали заявку (app_start): {app_start_new}")
print(f"Заполнили данные (data_fill): {data_fill_new}")
print(f"Конверсия: {conv_new:.1f}%")

print(f"\nСУЩЕСТВУЮЩИЕ клиенты:")
print(f"Начали заявку (app_start): {app_start_existing}")
print(f"Заполнили данные (data_fill): {data_fill_existing}")
print(f"Конверсия: {conv_existing:.1f}%")

print(f"\nРАЗНИЦА:")
print(f"Новые vs Существующие: {conv_new:.1f}% vs {conv_existing:.1f}%")
print(f"Существующие лучше на: {conv_existing - conv_new:.1f} процентных пунктов")
print(f"Относительно: в {conv_existing/conv_new:.1f} раза лучше")


ШАГ 3.3: КОНВЕРСИЯ НА ЭТАПЕ app_start → data_fill

НОВЫЕ клиенты:
Начали заявку (app_start): 826
Заполнили данные (data_fill): 321
Конверсия: 38.9%

СУЩЕСТВУЮЩИЕ клиенты:
Начали заявку (app_start): 1830
Заполнили данные (data_fill): 1183
Конверсия: 64.6%

РАЗНИЦА:
Новые vs Существующие: 38.9% vs 64.6%
Существующие лучше на: 25.8 процентных пунктов
Относительно: в 1.7 раза лучше


**Гипотеза продукт-менеджера:
«Новые клиенты не доверяют сложной форме и бросают процесс на этапе заполнения данных»**

## **4. Статистическая проверка гипотезы**

**Выбор теста:** Z-тест для сравнения двух пропорций (conversion rate)
**Почему Z-тест:** Сравниваем доли (проценты) двух независимых групп
**Уровень значимости:** α = 0.05

**Гипотезы:**
Гипотезы:
- H₀ (нулевая): Конверсия новых ≥ Конверсии существующих
  (новые клиенты НЕ ХУЖЕ существующих)
- H₁ (альтернативная): Конверсия новых < Конверсии существующих
  (новые клиенты ХУЖЕ существующих)

Направление теста: односторонний (left-tailed)

**Односторонний тест:** Проверяем именно "хуже", а не просто "разные"

In [20]:
# Статистическая проверка
from statsmodels.stats.proportion import proportions_ztest

# Данные: новые [успехи, испытания], существующие [успехи, испытания]
count = [321, 1183]  # заполнили данные
nobs = [826, 1830]   # начали заявку

# Односторонний Z-тест (alternative='smaller' - новые хуже)
z_stat, p_value = proportions_ztest(count, nobs, alternative='smaller')

print(f"Z-статистика: {z_stat:.4f}")
print(f"P-value: {p_value:.6f}")
print(f"Статистически значимо (p < 0.05): {p_value < 0.05}")

Z-статистика: -12.4111
P-value: 0.000000
Статистически значимо (p < 0.05): True


**Интерпретация:** P-value ≈ 0 < 0.05 → Отвергаем H₀. Новые клиенты статистически значимо хуже заполняют форму.